In [2]:
import pandas as pd
import numpy as np
from sklearn import svm, datasets

In [3]:
df_train = pd.read_csv('../Data/train.csv', header=None)
df_test  = pd.read_csv('../Data/test.csv', header=None)
df_train_Labels  = pd.read_csv('../Data/trainLabels.csv', header=None)

#### SVM

Since SVM can deal with high-dimensional data, I use SVM. I divided the data set into train (80%) and test set(20%). 

I predict the label of the test set based on train set. I repeated this process 10000 times and see the accuracy.



In [10]:
def SVM_accuracy(ker):
    accuracy=0
    iteration=100
    for i in range(iteration):
        msk = np.random.rand(len(df_train)) < 0.8

        # train
        train = df_train[msk]
        train_Labels= df_train_Labels[msk]

        # test
        test = df_train[~msk]
        test_Labels= df_train_Labels[~msk]

        
        svc = svm.SVC(kernel=ker, C=1,gamma='auto').fit(train, np.array(train_Labels[0]))

        test_matrix =  test.as_matrix()
        actual=list(test_Labels[0])
        prediction=[]
        for sample in test_matrix :
            prediction.append(svc.predict([sample])[0])
        count=0
        for ith in range(len(prediction)):
            if prediction[ith]==actual[ith]:
                count=count+1

        accuracy=accuracy+float(count)/len(test_Labels)
    print ker, accuracy/iteration

In [11]:
kernels=['linear', 'poly', 'rbf']

In [12]:
for kernel in kernels:
    SVM_accuracy(kernel)

linear 0.810350251026
poly 0.874322165269
rbf 0.910255204785


### Result:

rbf is the best

### Predict for submission 

Use rbf

In [17]:
svc_all = svm.SVC(kernel='rbf', C=1,gamma='auto').fit(df_train, np.array(df_train_Labels[0]))

In [18]:
df_test_matrix=df_test.as_matrix()

In [19]:
prediction=[]
for ith in range(len(df_test_matrix)):
    prediction.append([ith+1,svc_all.predict([df_test_matrix[ith]])[0]])

In [20]:
predicted=pd.DataFrame(prediction,columns=['Id','Solution'])

In [22]:
predicted=predicted.set_index('Id')

In [23]:
predicted.to_csv('../predicted_data/svm_outcome.csv')